# Text generation using RNN - Character Level

To generate text using RNN, we need a to convert raw text to a supervised learning problem format.

Take, for example, the following corpus:

"Her brother shook his head incredulously"

First we need to divide the data into tabular format containing input (X) and output (y) sequences. In case of a character level model, the X and y will look like this:

|      X     |  Y  |
|------------|-----|
|    Her b   |  r  |
|    er br   |  o  |
|    r bro   |  t  |
|     brot   |  h  |
|    broth   |  e  |
|    .....   |  .  |
|    .....   |  .  |
|    ulous   |  l  |
|    lousl   |  y  |

Note that in the above problem, the sequence length of X is five characters and that of y is one character. Hence, this is a many-to-one architecture. We can, however, change the number of input characters to any number of characters depending on the type of problem.

A model is trained on such data. To generate text, we simply give the model any five characters using which it predicts the next character. Then it appends the predicted character to the input sequence (on the extreme right of the sequence) and discards the first character (character on extreme left of the sequence). Then it predicts again using the new sequence and the cycle continues until a fix number of iterations. An example is shown below:

Seed text: "incre"

|      X                                            |  Y                       |
|---------------------------------------------------|--------------------------|
|                        incre                      |    < predicted char 1 >  |
|               ncre < predicted char 1 >              |    < predicted char 2 >  |
|       cre< predicted char 1 > < predicted char 2 >   |    < predicted char 3 >  |
|       re< predicted char 1 >< predicted char 2 > < predicted char 3 >   |    < predicted char 4 >  |
|                      ...                          |            ...           |

# Notebook Overview
1. Preprocess data
2. LSTM model
3. Generate code

In [3]:
# import libraries
import warnings
warnings.filterwarnings("ignore")

import os
import re
import numpy as np
import random
import sys
import io
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import get_file

# 1. Preprocess data

We're going to build a C code generator by training an RNN on a huge corpus of C code (the linux kernel code). You can download the C code used as source text from the following link:
https://github.com/torvalds/linux/tree/master/kernel

We have already downloaded the entire kernel folder and stored in a local directory

## Load C code

In [7]:
import os

# Set path where C files reside (make sure the path is correct and fully specified)
path = r"M:/Professional/MachineLearningAndArtificialIntelligence_Course/27_Recurrent_Neural_Networks/Generating_C_Code/linux-master/linux-master/kernel"

# Change to the target directory
if os.path.exists(path):
    os.chdir(path)
else:
    print(f"Path {path} does not exist.")

# List all files in the directory
file_names = os.listdir()
print(file_names)

['.gitignore', 'acct.c', 'async.c', 'audit.c', 'audit.h', 'auditfilter.c', 'auditsc.c', 'audit_fsnotify.c', 'audit_tree.c', 'audit_watch.c', 'backtracetest.c', 'bounds.c', 'bpf', 'capability.c', 'cfi.c', 'cgroup', 'compat.c', 'configs', 'configs.c', 'context_tracking.c', 'cpu.c', 'cpu_pm.c', 'crash_core.c', 'crash_reserve.c', 'cred.c', 'debug', 'delayacct.c', 'dma', 'dma.c', 'elfcorehdr.c', 'entry', 'events', 'exec_domain.c', 'exit.c', 'exit.h', 'extable.c', 'fail_function.c', 'fork.c', 'freezer.c', 'futex', 'gcov', 'gen_kheaders.sh', 'groups.c', 'hung_task.c', 'iomem.c', 'irq', 'irq_work.c', 'jump_label.c', 'kallsyms.c', 'kallsyms_internal.h', 'kallsyms_selftest.c', 'kallsyms_selftest.h', 'kcmp.c', 'Kconfig.freezer', 'Kconfig.hz', 'Kconfig.kexec', 'Kconfig.locks', 'Kconfig.preempt', 'kcov.c', 'kcsan', 'kexec.c', 'kexec_core.c', 'kexec_elf.c', 'kexec_file.c', 'kexec_internal.h', 'kheaders.c', 'kprobes.c', 'ksyms_common.c', 'ksysfs.c', 'kthread.c', 'latencytop.c', 'livepatch', 'locking'

In [8]:
# set path where C files reside

path = r"M:/Professional/MachineLearningAndArtificialIntelligence_Course/27_Recurrent_Neural_Networks/Generating_C_Code/linux-master/linux-master/kernel"

os.chdir(path)

file_names = os.listdir()
print(file_names)

['.gitignore', 'acct.c', 'async.c', 'audit.c', 'audit.h', 'auditfilter.c', 'auditsc.c', 'audit_fsnotify.c', 'audit_tree.c', 'audit_watch.c', 'backtracetest.c', 'bounds.c', 'bpf', 'capability.c', 'cfi.c', 'cgroup', 'compat.c', 'configs', 'configs.c', 'context_tracking.c', 'cpu.c', 'cpu_pm.c', 'crash_core.c', 'crash_reserve.c', 'cred.c', 'debug', 'delayacct.c', 'dma', 'dma.c', 'elfcorehdr.c', 'entry', 'events', 'exec_domain.c', 'exit.c', 'exit.h', 'extable.c', 'fail_function.c', 'fork.c', 'freezer.c', 'futex', 'gcov', 'gen_kheaders.sh', 'groups.c', 'hung_task.c', 'iomem.c', 'irq', 'irq_work.c', 'jump_label.c', 'kallsyms.c', 'kallsyms_internal.h', 'kallsyms_selftest.c', 'kallsyms_selftest.h', 'kcmp.c', 'Kconfig.freezer', 'Kconfig.hz', 'Kconfig.kexec', 'Kconfig.locks', 'Kconfig.preempt', 'kcov.c', 'kcsan', 'kexec.c', 'kexec_core.c', 'kexec_elf.c', 'kexec_file.c', 'kexec_internal.h', 'kheaders.c', 'kprobes.c', 'ksyms_common.c', 'ksysfs.c', 'kthread.c', 'latencytop.c', 'livepatch', 'locking'

In [9]:
# use regex to filter .c files
import re
c_names = ".*\.c$"

c_files = list()

for file in file_names:
    if re.match(c_names, file):
        c_files.append(file)

print(c_files)

['acct.c', 'async.c', 'audit.c', 'auditfilter.c', 'auditsc.c', 'audit_fsnotify.c', 'audit_tree.c', 'audit_watch.c', 'backtracetest.c', 'bounds.c', 'capability.c', 'cfi.c', 'compat.c', 'configs.c', 'context_tracking.c', 'cpu.c', 'cpu_pm.c', 'crash_core.c', 'crash_reserve.c', 'cred.c', 'delayacct.c', 'dma.c', 'elfcorehdr.c', 'exec_domain.c', 'exit.c', 'extable.c', 'fail_function.c', 'fork.c', 'freezer.c', 'groups.c', 'hung_task.c', 'iomem.c', 'irq_work.c', 'jump_label.c', 'kallsyms.c', 'kallsyms_selftest.c', 'kcmp.c', 'kcov.c', 'kexec.c', 'kexec_core.c', 'kexec_elf.c', 'kexec_file.c', 'kheaders.c', 'kprobes.c', 'ksyms_common.c', 'ksysfs.c', 'kthread.c', 'latencytop.c', 'module_signature.c', 'notifier.c', 'nsproxy.c', 'padata.c', 'panic.c', 'params.c', 'pid.c', 'pid_namespace.c', 'profile.c', 'ptrace.c', 'range.c', 'reboot.c', 'regset.c', 'relay.c', 'resource.c', 'resource_kunit.c', 'rseq.c', 'scftorture.c', 'scs.c', 'seccomp.c', 'signal.c', 'smp.c', 'smpboot.c', 'softirq.c', 'stackleak.c

In [10]:
# load all c code in a list
full_code = list()
for file in c_files:
    code = open(file, "r", encoding='utf-8')
    full_code.append(code.read())
    code.close()

In [11]:
# let's look at how a typical C code looks like
print(full_code[20])

// SPDX-License-Identifier: GPL-2.0-or-later
/* delayacct.c - per-task delay accounting
 *
 * Copyright (C) Shailabh Nagar, IBM Corp. 2006
 */

#include <linux/sched.h>
#include <linux/sched/task.h>
#include <linux/sched/cputime.h>
#include <linux/sched/clock.h>
#include <linux/slab.h>
#include <linux/taskstats.h>
#include <linux/sysctl.h>
#include <linux/delayacct.h>
#include <linux/module.h>

DEFINE_STATIC_KEY_FALSE(delayacct_key);
int delayacct_on __read_mostly;	/* Delay accounting turned on/off */
struct kmem_cache *delayacct_cache;

static void set_delayacct(bool enabled)
{
	if (enabled) {
		static_branch_enable(&delayacct_key);
		delayacct_on = 1;
	} else {
		delayacct_on = 0;
		static_branch_disable(&delayacct_key);
	}
}

static int __init delayacct_setup_enable(char *str)
{
	delayacct_on = 1;
	return 1;
}
__setup("delayacct", delayacct_setup_enable);

void delayacct_init(void)
{
	delayacct_cache = KMEM_CACHE(task_delay_info, SLAB_PANIC|SLAB_ACCOUNT);
	delayacct_tsk_init(&init_t

In [12]:
# merge different c codes into one big c code
text = "\n".join(full_code)
print("Total number of characters in entire code: {}".format(len(text)))

Total number of characters in entire code: 2243920


In [13]:
# top_n: only consider first top_n characters and discard the rest for memory and computational efficiency
top_n = 400000
text = text[:top_n]

## Convert characters to integers

In [15]:
# create character to index mapping
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [16]:
print("Vocabulary size: {}".format(len(chars)))

Vocabulary size: 95


## Divide data in input (X) and output (y)

### Create sequences

In [19]:
# define length for each sequence
MAX_SEQ_LENGTH = 50          # number of input characters (X) in each sequence 
STEP           = 3           # increment between each sequence
VOCAB_SIZE     = len(chars)  # total number of unique characters in dataset

sentences  = []              # X
next_chars = []              # y

for i in range(0, len(text) - MAX_SEQ_LENGTH, STEP):
    sentences.append(text[i: i + MAX_SEQ_LENGTH])
    next_chars.append(text[i + MAX_SEQ_LENGTH])

In [20]:
print('Number of training samples: {}'.format(len(sentences)))

Number of training samples: 133317


## Create input and output using the created sequences

When you're not using the Embedding layer of the Keras as the very first layer, you need to convert your data in the following format:
#### input shape should be of the form :  (#samples, #timesteps, #features)
#### output shape should be of the form :  (#samples, #timesteps, #features)

![Tensor shape](./jupyter resources/rnn_tensor.png)

#samples: the number of data points (or sequences)
#timesteps: It's the length of the sequence of your data (the MAX_SEQ_LENGTH variable).
#features: Number of features depends on the type of problem. In this problem, #features is the vocabulary size, that is, the dimensionality of the one-hot encoding matrix using which each character is being represented. If you're working with **images**, features size will be equal to: (height, width, channels), and the input shape will be (#training_samples, #timesteps, height, width, channels)

In [22]:
# create X and y
X = np.zeros((len(sentences), MAX_SEQ_LENGTH, VOCAB_SIZE), dtype=np.bool_)
y = np.zeros((len(sentences), VOCAB_SIZE), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [23]:
print("Shape of X: {}".format(X.shape))
print("Shape of y: {}".format(y.shape))

Shape of X: (133317, 50, 95)
Shape of y: (133317, 95)


Here, X is reshaped to (#samples, #timesteps, #features). We have explicitly mentioned the third dimension (#features) because we won't use the Embedding() layer of Keras in this case since there are only 97 characters. Characters can be represented as one-hot encoded vector. There are no word embeddings for characters.

# 2. LSTM

In [37]:
# define model architecture - using a two-layer LSTM with 128 LSTM cells in each layer
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_SEQ_LENGTH, VOCAB_SIZE), return_sequences=True, dropout=0.01))
model.add(LSTM(128, dropout=0.01))
model.add(Dense(VOCAB_SIZE, activation = "softmax"))

optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics = ['acc'])

In [39]:
# check model summary
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 50, 128)        │       114,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 95)             │        12,255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 258,527 (1009.87 KB)

 Trainable params: 258,527 (1009.87 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# fit model
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 105s 96ms/step - acc: 0.2459 - loss: 2.9260
Epoch 2/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 119s 115ms/step - acc: 0.5218 - loss: 1.7496
Epoch 3/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 122s 117ms/step - acc: 0.5862 - loss: 1.4833
Epoch 4/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 110s 106ms/step - acc: 0.3580 - loss: 2.5400
Epoch 5/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 107s 103ms/step - acc: 0.3746 - loss: 2.4477
Epoch 6/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 113s 108ms/step - acc: 0.1308 - loss: 3.4038
Epoch 7/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 111s 107ms/step - acc: 0.1785 - loss: 3.1880
Epoch 8/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 120s 116ms/step - acc: 0.2138 - loss: 3.0182
Epoch 9/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 123s 118ms/step - acc: 0.2326 - loss: 2.9317
Epoch 10/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 126s 121ms/step - acc: 0.2459 - loss: 2.8693
Epoch 11/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 136s 130ms/step - acc: 0.2462 - loss: 2.8697
Epoch 12/20
1042/1042 ━━━━━━━━━━━━━━━━━━━━

# 3. Generate code

Create a function that will make next character predictions based on temperature. If temperature is greater than 1, the generated characters will be more versatile and diverse. On the other hand, if temperature is less than one, the generated characters will be much more conservative.

In [43]:
# define function to sample next word from a probability array based on temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [45]:
np.random.multinomial(10, [0.05, 0.9, 0.05], size=2)

array([[ 0, 10,  0],
       [ 0,  9,  1]])

In [47]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random code to start text generation

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

-------------------------------------------------- diversity: 0.5
----- Generating with seed: "ntext save/restore which shouldn't be interrupted."
ntext save/restore which shouldn't be interrupted.
					aa>coatunt_cocrace coctactet & *  * Af ruat cactetaon all cot ent to coccle  *

														}

																												tant_speat AUAAT_AAADIT_OAT_AUAT;

																						 * sotes(AADEEIT, * * * AEEIT_AAALD_RAT_AAP_AAAVAT_TL_SUUET_SHATIT *  * * = ufaate che cousofacgoc retee atetes cet fen vouto  
																																																						  * *  *								ac (pe *toterure AUAAT_OAT_AAUSAAD *    ac cauteget ttost ac are: costunt cetart ac bent coct chochace ratoet stet_etemebit fate *		
																									cack coccoctectet chact *aces_ttos_cpots_to(>
														stactet;
		ac = rut_stals *artate arditet att nate coct
		= * *														 *													artectactunt ctetrate *  aate cactachetnutacs

						}
											ctero*   *  * * * A														htate ra

In [ ]:
# generate code

start_index = random.randint(0, len(text) - MAX_SEQ_LENGTH - 1) # pick random seed

for diversity in [0.5, 1.0, 1.5]:
        print('-'*50, 'diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + MAX_SEQ_LENGTH]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, MAX_SEQ_LENGTH, VOCAB_SIZE))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()